In [146]:
import numpy as np
import glob, os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import pandas as pd

from pathlib import Path
import librosa
import scipy

## Defined Functions:

In [147]:
def extract_features(signal, fs):
    # Always Need to give an input for sampling rate so that other higher level features can be extracter
   
    zcr = librosa.feature.zero_crossing_rate(signal); #Zero Crossing Rate
    spec_cent = librosa.feature.spectral_centroid(signal,fs);  #Spectral Centroid
    tontz = librosa.feature.tonnetz(signal,fs) #tonal centroid features
    S = librosa.feature.melspectrogram(signal,fs) # Mel Spectorgram
    mfc = librosa.feature.mfcc(signal,fs,S) #Mel frequency Coefficents
    return zcr, spec_cent, tontz, S, mfc
  
    
## Function to load all .wav files in a given folder to an array of (data, sample rate) ##
def load_folder(data_path):
    # input string of path to specific folder
    samples = [];
    
    for file in glob.glob(os.path.join(data_path,'*.wav')):
        temp,sr = librosa.load(file);
        temp = librosa.util.fix_length(temp,2*sr);
        samples.append([temp,sr])
        
    return samples
            

## Load Sample Files:

In [148]:
# CHANGE PATHS TO YOUR FOLDERS
kick_path = 'audio\kickSamples'; 
snare_path = 'audio\snareSamples';

kickSamples = load_folder(kick_path);
snareSamples = load_folder(snare_path);


In [149]:
np.shape(kickSamples) ## If files loaded correctly should return (# samples, 2)

(120, 2)

## Extract Features:

In [150]:
kickFeat = [extract_features(x[0], x[1]) for x in kickSamples]
snareFeat = [extract_features(x[0], x[1]) for x in snareSamples]

$\textbf{Features output as lists of feature attributes for each sample:}$

kickFeat[k][0] -> zcr features for kick sample k

kickFeat[k][1] -> spec centroid for kick sample k

kickFeat[k][2] -> tonal centroid for kick sample k

kickFeat[k][3] -> mel spec for kick smaple k

kickFeat[k][4] -> mfcs for kick sample k


In [151]:
# Organize extractions by feature classes rater than samples

# assign binary tags for sample types
class_tags = np.ones(len(kickSamples)) 
class_tags = np.append(class_tags, np.zeros(len(snareSamples)))

zero_co= []
cent_freq = []
tonal_ctroid = []
mfccs = []

i = 0
for s in kickFeat:
    zero_co.append(s[0][i])
    cent_freq.append( s[1][i])
    tonal_ctroid.append(s[2][i])
    mfccs.append(s[4][i])


for s in snareFeat:
    zero_co.append(s[0][i])
    cent_freq.append( s[1][i])
    tonal_ctroid.append(s[2][i])
    mfccs.append(s[4][i])

    
zero_co = (zero_co, class_tags)
cent_freq = (cent_freq,  class_tags)
tonal_ctroid = (tonal_ctroid, class_tags)
mfccs = (mfccs,  class_tags)

# Create dataset from extracted features

datasets = [make_moons(noise=0.3),make_circles(noise=0.3),zero_co,cent_freq,tonal_ctroid,mfccs]
h = .02  # step size in the mesh

## Classification:


In [152]:

h = .02  # step size in the mesh

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    # Classifiers are configured here
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]






## Perprocessing breakdown ##
i = 0 
analysis_data = np.zeros((len(datasets),len(names)))
for ds in datasets:
    #ds is a tuple
    # unpacks X as the feature training set, y as the target values
    X, y = ds
    # StandardScaler will remove the mean and scaling to unit variance
    # fit_transform: fits then transforms
    # fit: Compute the mean and std to be used for later scaling
    # transform: Perform standardization by centering and scaling
    X = StandardScaler().fit_transform(X)
    # splits the features into random train and test data
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.4, random_state=42)

    j=0
    for name, clf in zip(names, classifiers):
        clf.fit(X_train,y_train) #trains model
        score = clf.score(X_test, y_test) # tests model
        analysis_data[i][j] = score
        # clf may be the model, so it will need to be saved somewhere if it is accurate
        j+=1
    i+=1
        
df=pd.DataFrame(analysis_data,columns = names)
df.index= ['make_moons', 'make_circles','zero_crossing','centrial_frequency','tonnetz','mfccs']

C:\Users\anna_\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\anna_\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\anna_\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpc.py:430: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.25153769,  1.05515842]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 111, 'nit': 13, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\Users\anna_\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the opti

## Display Analysis of Results:

In [153]:
df

,Nearest Neighbors,Linear SVM,RBF SVM,Gaussian Process,Decision Tree,Random Forest,Neural Net,AdaBoost,Naive Bayes,QDA
make_moons,0.900000,0.800000,0.925000,0.825000,0.875000,0.800000,0.850000,0.825000,0.825000,0.825000
make_circles,0.325000,0.475000,0.325000,0.325000,0.400000,0.400000,0.375000,0.525000,0.450000,0.400000
zero_crossing,0.969388,0.938776,0.979592,0.989796,1.000000,0.989796,0.959184,0.979592,0.938776,0.989796
centrial_frequency,0.989796,0.989796,0.969388,0.989796,1.000000,1.000000,0.989796,1.000000,0.969388,0.969388
tonnetz,0.704082,0.663265,0.500000,0.775510,0.826531,0.836735,0.816327,0.908163,0.663265,0.979592
mfccs,1.000000,1.000000,0.969388,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.989796
